## Twitter US Airline Sentiment Analysis

Given Twitter US Airline Sentiment Dataset, which contains data for over 14000 tweets, your task is to predict the sentiment of the tweet i.e. positive, negative or neutral.
You are given:
1. A Training dataset csv file with X train and Y train data
2. A X test File and you have to predict and submit predictions for this file.
Read Instructions carefully -
1. Files are in csv format.
2. Submit a csv file with only predictions for X test data. File should not have any headers and should only have one column i.e. predictions. 
3. Submit your ipynb file as well.
4. Your score is based on number of accurate predictions.

In [1]:
import numpy as np
import pandas as pd 

training_data=pd.read_csv('/Users/shabeggill/Downloads/0000000000002747_training_twitter_x_y_train.csv')
testing_data=pd.read_csv('/Users/shabeggill/Downloads/0000000000002747_test_twitter_x_test.csv')

In [2]:
training_data

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [3]:
categories=training_data['airline_sentiment'].unique()
categories
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shabeggill/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
training_data['text']=training_data['text'].astype('string')

In [5]:
documents=[]
for i in range(len(training_data)):
    documents.append((word_tokenize(training_data['text'][i]),training_data['airline_sentiment'][i]))
documents      

[(['@',
   'SouthwestAir',
   'I',
   'am',
   'scheduled',
   'for',
   'the',
   'morning',
   ',',
   '2',
   'days',
   'after',
   'the',
   'fact',
   ',',
   'yes',
   '..',
   'not',
   'sure',
   'why',
   'my',
   'evening',
   'flight',
   'was',
   'the',
   'only',
   'one',
   'Cancelled',
   'Flightled'],
  'negative'),
 (['@',
   'SouthwestAir',
   'seeing',
   'your',
   'workers',
   'time',
   'in',
   'and',
   'time',
   'out',
   'going',
   'above',
   'and',
   'beyond',
   'is',
   'why',
   'I',
   'love',
   'flying',
   'with',
   'you',
   'guys',
   '.',
   'Thank',
   'you',
   '!'],
  'positive'),
 (['@',
   'united',
   'Flew',
   'ORD',
   'to',
   'Miami',
   'and',
   'back',
   'and',
   'had',
   'great',
   'crew',
   ',',
   'service',
   'on',
   'both',
   'legs',
   '.',
   'THANKS'],
  'positive'),
 (['@',
   'SouthwestAir',
   '@',
   'dultch97',
   'that',
   "'s",
   'horse',
   'radish',
   '😤🐴'],
  'negative'),
 (['@',
   'united',
   's

In [6]:
#POS gave me a number of tags but I want to get to a simpler pos tag and so we define a function for the same
#For finding pos_tags and converting them into simpler tags for lemmatization
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'): #if it starts with 'J'
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [7]:
#Getting stopwords
from nltk.corpus import stopwords
stop=stopwords.words('english') #getting english stopwords only

#Adding punctuations to list of stopwords
import string
punctuations=string.punctuation #this is a string
stop=stop+list(punctuations) #punctuations get added to the list of stopwords (only lists can be added)
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [8]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
lemmatizer=WordNetLemmatizer()
#Now I want to clean each of these files. We need to remove stopwords and do lammetization
#Creating a function for the same
def clean_review(words):
    output_words=[]
    for w in words: #for each word in words
        if w.lower() not in stop: #Need to check if the word is not in stopwords
            pos=pos_tag([w]) #Passing the word in the pos_tag as it is, is important, else you'll lose some information
            #We found the part of speech. Doing as done above 
            
            #Now lemmatize it
            clean_word=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1])) #Part of speech cannot be passed directly as done before.
            #pos='a' signifies adjective but pos_tags work differently. Thus we'll do get_simple_pos()
            #pos[0][1] gives the 2nd entry of the 0th tuple i.e. the pos_tag as in the case of [('better', 'RBR')], 
            # it would give 'RBR'
            
            output_words.append(clean_word.lower()) #converting to lower case and appending in the array
    return output_words

In [9]:
#Going through the documents one by one
#we pass the function clean_review() and category as entries in the tuple now and iterate for each document
documents=[(clean_review(document),category)for document,category in documents]

In [10]:
#To get to the categories
categories=[category for document,category in documents]

text_documents=[" ".join(document) for document,category in documents] #document is an array of words
#after applying join it will get converted to a single sentence as done above

In [11]:
x_train=text_documents
y_train=categories

In [12]:
testing_data['text']=testing_data['text'].astype('string')

In [13]:
documents_test=[]
for i in range(len(testing_data)):
    documents_test.append(word_tokenize(testing_data['text'][i]))
documents_test

[['@',
  'AmericanAir',
  'In',
  'car',
  'gng',
  'to',
  'DFW',
  '.',
  'Pulled',
  'over',
  '1hr',
  'ago',
  '-',
  'very',
  'icy',
  'roads',
  '.',
  'On-hold',
  'with',
  'AA',
  'since',
  '1hr',
  '.',
  'Ca',
  "n't",
  'reach',
  'arpt',
  'for',
  'AA2450',
  '.',
  'Wat',
  '2',
  'do',
  '?'],
 ['@',
  'AmericanAir',
  'after',
  'all',
  ',',
  'the',
  'plane',
  'didn',
  '’',
  't',
  'land',
  'in',
  'identical',
  'or',
  'worse',
  ')',
  'conditions',
  'at',
  'GRK',
  'according',
  'to',
  'METARs',
  '.'],
 ['@',
  'SouthwestAir',
  'ca',
  "n't",
  'believe',
  'how',
  'many',
  'paying',
  'customers',
  'you',
  'left',
  'high',
  'and',
  'dry',
  'with',
  'no',
  'reason',
  'for',
  'flight',
  'Cancelled',
  'Flightlations',
  'Monday',
  'out',
  'of',
  'BDL',
  '!',
  'Wow',
  '.'],
 ['@',
  'USAirways',
  'I',
  'can',
  'legitimately',
  'say',
  'that',
  'I',
  'would',
  'have',
  'rather',
  'driven',
  'cross',
  'country',
  'than',


In [14]:
#Going through the documents one by one
#we pass the function clean_review() and iterate for each document
documents_test=[(clean_review(document))for document in documents_test]

text_documents_test=[" ".join(document) for document in documents_test] #document is an array of words
#after applying join it will get converted to a single sentence as done above

In [15]:
x_test=text_documents_test

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec1=CountVectorizer(max_features=10000) #will pass say top 10000 words
x_train_features1=count_vec1.fit_transform(x_train)
x_test_features1=count_vec1.transform(x_test)

In [21]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(x_train_features1,y_train)
y_pred1=svc.predict(x_test_features1)

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(x_train_features1,y_train)
y_pred2=rf.predict(x_test_features1)

In [23]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
mnb.fit(x_train_features1,y_train)
y_pred3=mnb.predict(x_test_features1)

In [24]:
np.savetxt('Twitter_Sentiment_Analysis_SVM_Predictions.csv',y_pred1,delimiter=',',fmt='%s')

In [25]:
np.savetxt('Twitter_Sentiment_Analysis_RF_Predictions.csv',y_pred2,delimiter=',',fmt='%s')

In [26]:
np.savetxt('Twitter_Sentiment_Analysis_MNB_Predictions.csv',y_pred3,delimiter=',',fmt='%s')